In [8]:
import pandas as pd
import os

file_list = ["AlwaysTraf/csv dump/1.csv",
             "AlwaysTraf/csv dump/2.csv",
             "AlwaysTraf/csv dump/3.csv",
             "AlwaysTraf/csv dump/4.csv",
             "AlwaysTraf/csv dump/5.csv",
             "AlwaysTraf/csv dump/6.csv"]

patterns = ['ARP', 'TCP']
unique_values_protocol = []
lengths_always = []
end_length_list = []
#--------------------------------------------------------------------------------------------------
# методы для анализа нормальных дампов
#--------------------------------------------------------------------------------------------------
def first_round(): # считаем количество пакетов ARP протокола в нормальном трафике
    countARP = 0
    for pattern in patterns:
        filter_result = df[df['Protocol'].str.contains(pattern)]
        count = len(filter_result)
        if pattern == 'ARP':
            countARP += count
    return countARP

def second_round(): # записываем уникальные протоколы в нормальном трафике
    unique_values = df['Protocol'].unique()
    for value in unique_values:
        unique_values_protocol.append(value)

def get_max_length(df): # получаем максимальное значение объема пакета
    max_length = df['Length'].max()
    return max_length

for i, file in enumerate(file_list, start=1): #цикл для прохода по всем дампам нормального трафика
    df = pd.read_csv(file)
    first_round()
    second_round()
    max_len = get_max_length(df)
    lengths_always.append(max_len) # получаем максимальное значение в 1-6 дампе нормального трафика

end_length_list =  list(set(lengths_always)) # удаляем повторяющиеся значения максимального объема

def remove_duplicates(values): # удаляем повторяющиеся значения протоколов 
    unique_values = list(set(values))
    return unique_values

final_list = []
final_list.extend(unique_values_protocol)
unique_values = remove_duplicates(final_list)

print("Данные нормального трафика получены")
print("")

#--------------------------------------------------------------------------------------------------
# выявление аномалий.
# Проверка происходит по одному файлу. Для проверки другого - нужно установить соответствующее
# значение в 120-ой строке, (file_list_anomal[$])
#--------------------------------------------------------------------------------------------------

file_list_anomal = ["AnomalTraf/csv dump/6.csv",
                    "AnomalTraf/csv dump/11.csv",
                    "AnomalTraf/csv dump/8.csv",
                    "AnomalTraf/csv dump/10.csv"]

real_arp = 0
ddos_count = 0
ddos_porog = 100
prev_no = None
normalArms = first_round()
normalArms = normalArms - 10
protocol_values_list = []
anomal_pattern = []

DatFr = pd.read_csv(file_list_anomal[3])

ml_Datfr = pd.DataFrame(columns=DatFr.columns.tolist() + ['ML'])
DatFr['ML'] = 0

def find_length_anomalies(end_length_list, DatFr):  # поиск аномалий объема пакета
    for index, row in DatFr.iterrows():
        length = row['Length']
        for max_length in end_length_list:
            if max_length < length:
                protocol = row['Protocol']
                source = row['Source']
                destination = row['Destination']
                info = row['Info']
                print(f"Обнаружена аномалия: слишком большой пакет информации. Протокол: {protocol}, IP отправителя: {source}, IP получателя: {destination}, Значение: {info}, Размер: {length}")
                DatFr.at[index, 'ML'] = 1
                print("")

find_length_anomalies(end_length_list, DatFr)

def search_anomal_protocol():  # ищем протоколы в аномальном трафике
    protocol_values = DatFr['Protocol'].unique()
    for protocol in protocol_values:
        protocol_values_list.append(protocol)

search_anomal_protocol()

def compare_lists(unique_values, protocol_values_list):  # сравниваем списки протоколов: нормального и аномального трафика
    for pattern in protocol_values_list:
        if pattern not in unique_values:
            anomal_pattern.append(pattern)
    return anomal_pattern

anomal_pattern = compare_lists(unique_values, protocol_values_list)

compare_lists(unique_values, protocol_values_list)

def find_pattern_values(DatFr, anomal_pattern):  # если после сравнения нашлась разница, то считаем ее аномалией
    for index, row in DatFr.iterrows():
        for pattern in anomal_pattern:
            if pattern in row['Protocol']:
                print(f"Найден паттерн: {pattern}, IP отправителя:  {row['Source']}, IP получателя:  {row['Destination']}, Значение: {row['Info']}")
                DatFr.at[index, 'ML'] = 1
                print("")
                break

find_pattern_values(DatFr, anomal_pattern)

for index, row in DatFr.iterrows():  # вылавливаем новые подключения и DDoS атаки
    if row['Protocol'] == 'ARP':
        real_arp += 1
        if real_arp > normalArms * 2:
            print(f"Детектировано подключение неизвестного устройства.\nARP подключение")
            DatFr.at[index, 'ML'] = 1  # Записываем значение 1 в столбец 'ML'
    elif row['Protocol'] == 'TCP':  # Отлов Ddos
        # Поиск слова 'SYN' в столбце 'Info'
        if 'SYN' in row['Info']:
            # Запоминаем номер записи
            if prev_no is None:
                prev_no = row['No.']
            else:
                if (row['No.'] - prev_no) == 1:
                    ddos_count += 1
                    if ddos_count >= ddos_porog:
                        print(f"Детектирована DDoS атака\n{index + 1 - ddos_count}")
                        for i in range(ddos_count):
                            DatFr.loc[index - i, 'ML'] = 1
                        ddos_count = 0
                        
                prev_no = row['No.']
        else:
            ddos_count = 0

# Путь для сохранения CSV файла
csv_file_path = 'MlDump/result5.csv'

# Сохранение DataFrame в CSV файл
DatFr.to_csv(csv_file_path, index=False)

Данные нормального трафика получены

Обнаружена аномалия: слишком большой пакет информации. Протокол: DNS, IP отправителя: 192.168.10.8, IP получателя: 56.110.14.82, Значение: Unknown operation (8) 0x4141[Malformed Packet], Размер: 10042

Найден паттерн: RTSP, IP отправителя:  192.168.10.9, IP получателя:  192.168.10.8, Значение: Continuation

Найден паттерн: HTTP, IP отправителя:  192.168.10.5, IP получателя:  216.58.209.238, Значение: GET / HTTP/1.1 

Найден паттерн: HTTP, IP отправителя:  192.168.10.5, IP получателя:  216.58.209.238, Значение: GET / HTTP/1.1 

Найден паттерн: HTTP, IP отправителя:  192.168.10.5, IP получателя:  216.58.209.238, Значение: GET / HTTP/1.1 

Найден паттерн: HTTP, IP отправителя:  192.168.10.5, IP получателя:  216.58.209.238, Значение: GET / HTTP/1.1 

Найден паттерн: MQTT, IP отправителя:  192.168.10.9, IP получателя:  192.168.10.9, Значение: Publish Received (id=22616)

Найден паттерн: IPSICTL, IP отправителя:  192.168.10.5, IP получателя:  192.168.10.9